## Generate predictions under :
 - district
 - province
 - age group
 - profession
 

In [29]:
import numpy as np 
import pandas as pd 
import sklearn
import pickle

_general test data pre-processing_

In [30]:
minmax_scaler = pickle.load(open('../../../data/models/minMax_lsmt_1.pkl', 'rb'))

In [31]:
test_df = pd.read_csv('../../../data/main_data/final_test.csv')
test_df.sort_values(by=['index','formal_date'], inplace=True)
uniq_dist = test_df['index'].unique().tolist()
num_dists = len(uniq_dist)


In [32]:
dist_df_list = []
for i in uniq_dist:
    sub_df = test_df.loc[test_df['index'] == i, :]
    dist_df_list.append(sub_df)

In [33]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [34]:
def preProcess(df):
    df.drop('formal_date', axis=1, inplace=True)
    df_value = df.values
    df_value.astype('float32')
    scaled_df = minmax_scaler.transform(df_value)
    reframed_df = series_to_supervised(scaled_df, 1, 1)
    reframed_df = reframed_df.values
    test_X, test_y = reframed_df[:, :-1], reframed_df[:, -1]
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    return test_X



In [35]:
pre_processed_data = []
for t_df in dist_df_list:
    pre_processed_data.append(preProcess(t_df))

Predicting

In [36]:
from tensorflow.keras.models import load_model
model = load_model('../../../data/models/LSTM_1.h5')

In [38]:
yhat_list = []
for dat in pre_processed_data:
    yhat = model.predict(dat)
    yhat_list.append(yhat)

In [50]:
def inversePreProcess(df, test_x):
    test_X = test_x.reshape((test_x.shape[0], test_x.shape[2]))
    inv_yhat = np.concatenate((test_X[:, :14], yhat), axis=1)
    inv_yhat = minmax_scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-1]
    return inv_yhat


In [51]:
res_list = []
for i in range(0,len(yhat_list)):
    res_list.append(inversePreProcess(yhat_list[i], pre_processed_data[i]))


In [56]:
dist_result = [j[-1] for j in res_list]
dist_result

[0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905,
 0.036300468457582905]

In [ ]:
w